In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np

from wufam.data.prepare_data import read_kf_data
from wufam.config.trading_config import TradingConfig
from wufam.dataset import Dataset
from wufam.strategies.base_strategy import BaseStrategy
from wufam.backtest.rolling_backtest import run_rolling_backtest
from wufam.metrics.metrics import calc_sharpe

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_25_D
FACTORS_DATASET = Dataset.FACTORS_D
WEIGHTING = "value_weighted"
FACTOR_ANNUALIZE = 252

## 1.1

In [3]:
portfolios_total_r, portfolios_xs_r, factors_df, rf = read_kf_data(
    portfolios_filename=PATH / DATASET,
    factors_filename=PATH / FACTORS_DATASET,
    start_date=START,
    end_date=END,
    weighting=WEIGHTING,
)

In [4]:
assert (
    portfolios_total_r.shape[0]
    == portfolios_xs_r.shape[0]
    == factors_df.shape[0]
    == rf.shape[0]
)

## 1.2

In [5]:
from wufam.strategies.heuristics.equally_weighted import EWStrategy
from wufam.strategies.optimized.mean_var import MeanVariance
from wufam.strategies.optimized.min_var import MinVariance

trading_config = TradingConfig(total_exposure=1)

ew_strategy = EWStrategy()

In [6]:
from wufam.estimation.mean.sample_mu_estimator import SampleMuEstimator
from wufam.estimation.covariance.sample_cov_estimator import SampleCovEstimator

mv_strategy = MeanVariance(
    mu_estimator=SampleMuEstimator(),
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [7]:
min_var_strategy = MinVariance(
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [8]:
trading_config_min_var = TradingConfig(total_exposure=1, min_exposure=0.0)

min_var_c_strategy = MinVariance(
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config_min_var,
    window_size=365,
)

In [9]:
from wufam.estimation.covariance.shrinkage.lw_cv_cov_estimator import (
    LedoitWolfCVCovEstimator,
)

min_var_lw_strategy = MinVariance(
    cov_estimator=LedoitWolfCVCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [10]:
from wufam.estimation.covariance.shrinkage.qis import QISCovEstimator

min_var_qis_strategy = MinVariance(
    cov_estimator=QISCovEstimator(),
    trading_config=trading_config,
    window_size=365,
)

In [11]:
min_var_lw_cv_strategy = MinVariance(
    cov_estimator=LedoitWolfCVCovEstimator(alphas=np.linspace(0.0, 1.0, 100)),
    trading_config=trading_config,
    window_size=365,
)

In [12]:
from wufam.estimation.covariance.factor.factor_cov_estimator import FactorCovEstimator
from wufam.estimation.covariance.shrinkage.pca_cov_estimator import PCACovEstimator

factor_cov_estimator = FactorCovEstimator(
    factor_cov_estimator=SampleCovEstimator(),
    residual_cov_estimator=PCACovEstimator(),
    factors_selection=["Mkt-RF"],
)

min_var_factor_strategy = MinVariance(
    cov_estimator=factor_cov_estimator,
    trading_config=trading_config,
    window_size=365,
)

In [13]:
from wufam.strategies.timed.vol_managed_strategy import VolManagedStrategy

vol_managed_trading_config = TradingConfig(min_exposure=0.0, max_exposure=2.0)

vol_managed_strategy = VolManagedStrategy(
    trading_config=vol_managed_trading_config,
    vol_window=21,
)

In [14]:
def evaluate_strategy(
    strategy: BaseStrategy, return_weights: bool = False
) -> tuple[float, float]:
    total_r, turnover = run_rolling_backtest(
        strategy=strategy,
        excess_returns=portfolios_xs_r,
        factors=factors_df,
        rf=rf,
        freq="M",
        trading_lag=1,
    )

    sr = calc_sharpe(
        strategy_total_r=total_r,
        rf_rate=rf,
        factor_annualize=FACTOR_ANNUALIZE,
    )

    avg_turnover = turnover.iloc[1:].mean()

    return sr, avg_turnover

In [15]:
evaluate_strategy(ew_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:00<00:00, 1293.04it/s]


(0.44198649620311253, np.float64(0.017344865113044353))

In [16]:
evaluate_strategy(mv_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:01<00:00, 595.78it/s]


(0.023077991516703287, np.float64(186.51542103379933))

In [17]:
evaluate_strategy(min_var_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:01<00:00, 636.55it/s]


(0.9921328793030837, np.float64(1.316652275594975))

In [18]:
evaluate_strategy(min_var_c_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:05<00:00, 113.76it/s]


(0.719614518335151, np.float64(0.22562515871206587))

In [19]:
evaluate_strategy(min_var_lw_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:01<00:00, 541.30it/s]


(1.227326905475528, np.float64(0.9037496358078545))

In [20]:
evaluate_strategy(min_var_factor_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:16<00:00, 40.35it/s]


(1.1740884083611147, np.float64(0.9828498585878805))

In [21]:
evaluate_strategy(min_var_qis_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:02<00:00, 320.44it/s]


(1.0747897587797615, np.float64(1.5307430135354767))

In [22]:
evaluate_strategy(vol_managed_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:01<00:00, 608.67it/s]


(0.5398630805684842, np.float64(0.44095377787995244))

In [23]:
evaluate_strategy(min_var_lw_cv_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [03:08<00:00,  3.50it/s]


(1.2091045047694422, np.float64(0.9223255553409698))

In [24]:
min_var_lw_cv_strategy.cov_estimator.best_alpha

np.float64(0.05050505050505051)

In [25]:
trading_config_naive_min_var = TradingConfig(total_exposure=1, min_exposure=0.5 / 25)

min_var_naive_c_strategy = MinVariance(
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config_naive_min_var,
    window_size=None,
)

In [26]:
evaluate_strategy(min_var_naive_c_strategy)

Optimizing Strategy: 100%|██████████| 659/659 [00:05<00:00, 110.92it/s]


(0.5539918349659848, np.float64(0.028896835087247448))